In [37]:
# Heterogeneous pooling
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [38]:
# all the data will be stored in this dataframe, with the method name, mean accuracy, standard deviation, lower and upper bound
df = pd.DataFrame(columns=['method', 'mean', 'std', 'lower', 'upper'])



In [39]:
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')


# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test=scaler.transform(X_test)



In [44]:
# Zero Rule Baseline
from sklearn.dummy import DummyClassifier

clf = DummyClassifier(strategy='most_frequent')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

df = df.append({'method': 'ZR', 'mean': scores.mean(), 'std': scores.std(), 'lower': scores.mean() - scores.std(), 'upper': scores.mean() + scores.std()}, ignore_index=True)


/tmp/ipykernel_33107/1863649893.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'method': 'ZR', 'mean': scores.mean(), 'std': scores.std(), 'lower': scores.mean() - scores.std(), 'upper': scores.mean() + scores.std()}, ignore_index=True)


In [52]:
# Bagging with hyperparameter tuning
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline

bg = BaggingClassifier(n_estimators=3)

# three rounds of nested cycles of validation and testing, with intern cycle with 4 folds and the external with 10 folds
cv_inner = RepeatedStratifiedKFold(n_splits=4, n_repeats=3, random_state=1)
cv_outer = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)


pipe = Pipeline(steps=[('s', StandardScaler()), ('m', bg)])


params = {'m__n_estimators': [3, 9, 15, 21]}


scores = []
best_acc = 0
for train_ix, test_ix in cv_outer.split(X_train, y_train):
    # split data
    X_train_inner, X_test_inner = X_train.iloc[train_ix, :], X_train.iloc[test_ix, :]
    y_train_inner, y_test_inner = y_train.iloc[train_ix], y_train.iloc[test_ix]


    # define search
    search = GridSearchCV(pipe, param_grid=params, scoring='accuracy', cv=cv_inner, n_jobs=-1)

    rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

    # execute search
    result = cross_val_score(search, X_train_inner, y_train_inner, cv=rkf, n_jobs=-1)

    scores.append(result.mean())
    # check the best model
    if result.mean() > best_acc:
        best_acc = result.mean()
        best_model = search

df = pd.concat([df, pd.DataFrame({'method': ['Bagging'], 'mean': [np.mean(scores)], 'std': [np.std(scores)], 'lower': [np.mean(scores) - np.std(scores)], 'upper': [np.mean(scores) + np.std(scores)]})], ignore_index=True)




/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'